__Tools for grading portions of a super kick ass assignment__

created: 23 April 2020

In [ ]:
# Dynamically sets location so do not have to manually toggle between dropbox and documents
import os
path = "/".join([a for a in os.path.abspath("").split('/') if a not in ['Notebooks', 'personal']])
%cd $path

import pandas as pd
import os

#Plotting 
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")


from CanvasHacks import environment
from CanvasHacks.Configuration import InteractiveConfiguration
# from CanvasHacks.PeerReviewed.Definitions import TopicalAssignment, ContentAssignment
from CanvasHacks.Models.QuizModels import QuizData

# Requests
from CanvasHacks.Api.RequestTools import get_all_course_assignments, get_assignments_needing_grading
from CanvasHacks.Api.RequestTools import send_get_request, send_post_request, send_put_request, make_url, make_request_header
from CanvasHacks.Api.RequestTools import make_request_header, get_all_course_quizzes, get_all_quiz_submissions
import requests

from CanvasHacks.Api.UrlTools import make_url
from CanvasHacks.Api.UploadGradeTools import make_upload_button
import CanvasHacks.Api.DownloadProcessingTools as PT

# Repos
from CanvasHacks.Repositories.DataManagement import DataStore
from CanvasHacks.Repositories.factories import WorkRepositoryFactory, WorkRepositoryLoaderFactory
from CanvasHacks.Repositories.quizzes import QuizRepository, ReviewRepository
from CanvasHacks.Repositories.codes import AccessCodeRepo
from CanvasHacks.Repositories.reviewer_associations import assign_reviewers, AssociationRepository
from CanvasHacks.Repositories.students import StudentRepository
from CanvasHacks.Repositories.submissions import SubmissionRepository, QuizSubmissionRepository
# from CanvasHacks.Repositories.quizzes import process_work, load_student_work, remove_non_final_attempts, make_drop_list, save_json, drop_columns_from_frame


# Filesystem
from CanvasHacks.Files.FileTools import makeDataFileIterator, create_folder
from CanvasHacks.Files.QuizReportFileTools import load_new
# from CanvasHacks.QuizReportFileTools import  make_quiz_repo, load_activity_data_from_files

# Widgets
from CanvasHacks.Widgets.AssignmentSelection import make_assignment_chooser, view_selected_assignments, view_ungraded_assignments
from CanvasHacks.Widgets.LiveSelection import make_test_selector
from CanvasHacks.Widgets.AssignmentSelection import make_unit_chooser, make_selection_button
from CanvasHacks.Widgets.ConsolidatedTextOutput import make_assignment_header, make_consolidated_text_fields
from CanvasHacks.Widgets.InputFields import make_course_ids_input, make_canvas_token_input, make_canvas_url_input, make_general_reset_button


#Grading
from CanvasHacks.GradingHandlers.quiz import QuizGrader
# import CanvasHacks.GradingTools.nonempty as GT

In [ ]:
# !!!!!!!!!!!!!!!!!!!!
# This must be manually adjusted if going to give 
# quarter credit

# Use this date for things due before the midterm
# QUARTER_CREDIT_DATE =  '2019-10-16T07:59:00Z'

# Date for things due after the midterm
# QUARTER_CREDIT_DATE =  '2020-12-18T07:59:00Z'

ToDo

    make_assignment_chooser is not picking up all assignments needing grading

In [ ]:
# assigns = get_all_course_assignments( environment.CONFIG.course_ids[0] )
# len(assigns)
get_assignments_needing_grading(environment.CONFIG.course_ids[0])

In [ ]:
# dev hotfix workaround for F25
environment.CONFIG.set_unit(1)


assignments=[]
for course_id in environment.CONFIG.course_ids:
    assignments += get_assignments_needing_grading( course_id)

    # assignments += get_assignments_with_submissions( course_id, **kwargs )
assignments = [ (a[ 'id' ], a[ 'name' ]) for a in assignments ]


# Get quiz ids and names

In [ ]:
make_test_selector()
make_unit_chooser(num_units=8)

In [ ]:
make_assignment_chooser(needs_grading=False)

In [ ]:
from CanvasHacks.GradingMethods.nonempty import CreditForNonEmptyPoints
from CanvasHacks.GradingMethods.review import ReviewBasedPoints
from CanvasHacks.GradingMethods.wordcount import GradeByWordCountPoints

# unit 1
# REVIEW_COLUMNS = ["3868950: Having read the author's essay carefully, I feel it deserves this many points."]

# todo make a selector which let's choose without messing up formatting
# unit 2
# REVIEW_COLUMNS = ["""4082297: Having read the author's essay carefully, I feel it deserves this many points.
# [Remember, the essay grade is: 1 point for completion; 3 from the reviewer (what you're assigning here); and 3 from the instructor]"""]

# REVIEW_COLUMNS = ["""4082297: \nHaving read the author's essay carefully, I feel it deserves this many points.\n[Remember, the essay grade is: 1 point for completion; 3 from the reviewer (what you're assigning here); and 3 from the instructor]\n"""]

# This holds the selected column in which the reviewer assigns points
REVIEW_COLUMNS = []

# word count grading thresholds
thresholds = [
    {
        'min_count': 600,
        'points': 6
    },
     {
        'min_count': 500,
        'points': 5
    },
     {
        'min_count': 400,
        'points': 4
    },
    {
        'min_count': 300,
        'points': 3
    },
    
    {
        'min_count': 100,
        'points': 1
    },
]

cnep = CreditForNonEmptyPoints(max_possible_points=2)

# rbp = ReviewBasedPoints(
#     graded_activity=environment.CONFIG.unit.initial_work,
#     review_activity=environment.CONFIG.unit.review,
#     review_columns=REVIEW_COLUMNS,
#     max_possible_points=3)

wcp = GradeByWordCountPoints(max_possible_points=6, threshold_dicts=thresholds)

environment.CONFIG.unit.initial_work.grade_methods = [cnep,  wcp]


from IPython.display import display
from ipywidgets import widgets, VBox

def make_review_column_chooser(review_columns):
    def adder(item_id, item_name):
        review_columns.append(item_name)

    def getter():
        return review_columns 

    def remover(item_name):
        el = list( filter( lambda x: x[ 0 ] == item_name, review_columns ) )[ 0 ]
        idx = review_columns.index( el )
        return review_columns.pop( idx )

    cols = [c for c in rbp.review_repo.data.columns if len(c) >10]
    buttons = [make_selection_button(item_id=c, name=c, get_func=getter, add_func=adder, width='100%', remove_func=remover ) for c in cols]
    head = widgets.HTML( value="<h4>Select the question in which reviewers determine score</h4>" )             

    display( VBox( [VBox([head]), VBox(buttons)] ))

# dev CAN-72 Assign preliminary essay grade based on review

In [ ]:
# F20

from CanvasHacks.GradingMethods.nonempty import CreditForNonEmptyPoints
from CanvasHacks.GradingMethods.review import ReviewBasedPoints
from CanvasHacks.GradingMethods.wordcount import GradeByWordCountPoints

# unit 1
# REVIEW_COLUMNS = ["3868950: Having read the author's essay carefully, I feel it deserves this many points."]

# todo make a selector which let's choose without messing up formatting
# unit 2
# REVIEW_COLUMNS = ["""4082297: Having read the author's essay carefully, I feel it deserves this many points.
# [Remember, the essay grade is: 1 point for completion; 3 from the reviewer (what you're assigning here); and 3 from the instructor]"""]

# REVIEW_COLUMNS = ["""4082297: \nHaving read the author's essay carefully, I feel it deserves this many points.\n[Remember, the essay grade is: 1 point for completion; 3 from the reviewer (what you're assigning here); and 3 from the instructor]\n"""]

# This holds the selected column in which the reviewer assigns points
REVIEW_COLUMNS = []

# word count grading thresholds
thresholds = [
    {
        'min_count': 600,
        'points': 3
    },
    {
        'min_count': 300,
        'points': 2
    },
    {
        'min_count': 100,
        'points': 1
    },
]

cnep = CreditForNonEmptyPoints(max_possible_points=1)

rbp = ReviewBasedPoints(
    graded_activity=environment.CONFIG.unit.initial_work,
    review_activity=environment.CONFIG.unit.review,
    review_columns=REVIEW_COLUMNS,
    max_possible_points=3)

wcp = GradeByWordCountPoints(max_possible_points=3, threshold_dicts=thresholds)

environment.CONFIG.unit.initial_work.grade_methods = [cnep, rbp, wcp]


from IPython.display import display
from ipywidgets import widgets, VBox

def make_review_column_chooser(review_columns):
    def adder(item_id, item_name):
        review_columns.append(item_name)

    def getter():
        return review_columns 

    def remover(item_name):
        el = list( filter( lambda x: x[ 0 ] == item_name, review_columns ) )[ 0 ]
        idx = review_columns.index( el )
        return review_columns.pop( idx )

    cols = [c for c in rbp.review_repo.data.columns if len(c) >10]
    buttons = [make_selection_button(item_id=c, name=c, get_func=getter, add_func=adder, width='100%', remove_func=remover ) for c in cols]
    head = widgets.HTML( value="<h4>Select the question in which reviewers determine score</h4>" )             

    display( VBox( [VBox([head]), VBox(buttons)] ))

In [ ]:
make_review_column_chooser(REVIEW_COLUMNS)

In [ ]:
from CanvasHacks.executables.GradeAssignmentStep import GradeAssignment

WAIT =0

NO_LATE_PENALTY =True

UPLOAD_GRADES = True
# UPLOAD_GRADES = False

# hack because selector isn't picking up the essays
step = GradeAssignment(activity=environment.CONFIG.unit.initial_work, 
                       rest_timeout=WAIT, 
                       no_late_penalty=NO_LATE_PENALTY, 
                       upload_grades=UPLOAD_GRADES )
# step = GradeAssignment(rest_timeout=WAIT, no_late_penalty=NO_LATE_PENALTY, upload_grades=UPLOAD_GRADES )

step.run()

In [ ]:
step.workRepo.data

In [ ]:
step.grade_records[1].__dict__

# Grade (post-CAN-44)
## Grade review (quiz type)

### CAN-72 dev

In [ ]:


cnep = CreditForNonEmptyPoints(max_possible_points=1)

rbp = ReviewBasedPoints(
    graded_activity=environment.CONFIG.unit.initial_work,
    review_activity=environment.CONFIG.unit.review,
    review_columns=REVIEW_COLUMNS,
    max_possible_points=2)

environment.CONFIG.unit.review.grade_methods = [cnep]
environment.CONFIG.unit.review.corrections = [rbp]

make_review_column_chooser(REVIEW_COLUMNS)

In [ ]:
# This may have uploaded scores of 1 instead of 0!!!! for Unit 3 warm up


from CanvasHacks.executables.GradeQuizStep import GradeQuiz

WAIT =5
NO_LATE_PENALTY =True
UPLOAD_GRADES = True

step = GradeQuiz(rest_timeout=WAIT, 
                 no_late_penalty=NO_LATE_PENALTY, 
                 upload_grades=UPLOAD_GRADES )
#step.run()

### Pre CAN-72 grade quiz

In [ ]:
# This may have uploaded scores of 1 instead of 0!!!! for Unit 3 warm up


from CanvasHacks.executables.GradeQuizStep import GradeQuiz

WAIT =5
NO_LATE_PENALTY =True
UPLOAD_GRADES = True

step = GradeQuiz(rest_timeout=WAIT, 
                 no_late_penalty=NO_LATE_PENALTY, 
                 upload_grades=UPLOAD_GRADES )
#step.run()

In [ ]:
step.graded

## Grade assignment type activities

Essays

In [ ]:
from CanvasHacks.executables.GradeAssignmentStep import GradeAssignment

WAIT =0

NO_LATE_PENALTY =True
UPLOAD_GRADES = False

# hack because selector isn't picking up the essays
step = GradeAssignment(activity=environment.CONFIG.unit.initial_work, 
                       rest_timeout=WAIT, 
                       no_late_penalty=NO_LATE_PENALTY, 
                       upload_grades=UPLOAD_GRADES )
# step = GradeAssignment(rest_timeout=WAIT, no_late_penalty=NO_LATE_PENALTY, upload_grades=UPLOAD_GRADES )

step.run()

In [ ]:
step.graded

# ------------------- Dev ------------------- 

## grade on wordcount

In [ ]:
from CanvasHacks.GradingMethods.wordcount import GradeByWordCount, make_pct_required_count_thresholds


REQUIRED_CNT = 1000
thresholds = make_pct_required_count_thresholds(REQUIRED_CNT)
grader = GradeByWordCount(thresholds)

ddd = step1.work_repo.data.copy(deep=True)

ddd['grade'] = ddd.apply(lambda x: grader.grade(x.body), axis=1)

## Gather statistics for assessment

In [ ]:
# make and store wordbags

In [ ]:
activity = [c for c in environment.CONFIG.unit.components if c.id == environment.CONFIG.assignments[0][0]][0]

In [ ]:
# Load the characteristics of the quizzes that we are going to grade
qs = []
for s in environment.CONFIG.course_ids:
    qs += get_all_course_quizzes(s)
to_grade = [assign_id for assign_id, name in environment.CONFIG.assignments]
quizzes = [QuizData(**q) for q in qs if q['assignment_id'] in to_grade]

In [ ]:
# programmatically get data
workRepo = make_quiz_repo(environment.CONFIG.course, activity)

In [ ]:
workRepo = QuizRepository(, environment.CONFIG.course)


In [ ]:
workRepo.points_per_question

In [ ]:
environment.CONFIG.unit.topical_assignment

# Make folders to store quiz data

In [ ]:
create_folder(environment.CONFIG.unit.topical_assignment.folder_path)
# for q in quizzes:
#     create_folder(q.folder_path)

# Working on CAN-41

In [ ]:

def load_activity_data_from_files(activity, course):
    """Get complete set of data for activity from a bunch of 
    potentially inconsistent files. 
    Created in CAN-41
    Loads the data into a repository object of the appropriate type
    """
    fiter = makeDataFileIterator(activity.folder_path)
    frames = []
    try:
        while True:
            f = next(fiter)
            print("loading: ", f)
            f = pd.read_csv(f)
            if 'student_id' not in f.columns:
                f.rename( { 'id': 'student_id' }, axis=1, inplace=True )
            frames.append(f)
    except StopIteration:
        print("Loaded data from {} files".format(len(frames)))

        data = pd.concat(frames, sort=True)
        # drop_columns_from_frame(f)
        # f.drop(EXTRANEOUS_COLUMNS, axis=1, inplace=True)
        # repo = ReviewRepository(activity)

        print("Loaded {} rows from all files in folder".format(len(f)))

        # this shouldn't be necessary in future. Only when were saving sheet plus submissions
        if 'score_x' in data.columns:
            def fix_score(row):
                return row[['score', 'score_x', 'score_y']].mean()
            data.score = data.apply(lambda x: fix_score(x), axis=1)
            data.drop([ 'score_x', 'score_y'], axis=1, inplace=True)

        repo = WorkRepositoryFactory.make(activity, course)
        repo.set_question_columns(data)
        drop_columns_from_frame(data)

        # f = f[ repo.keep_columns ]
        data.drop_duplicates(inplace=True)
        repo.data = data
        print("{} rows loaded to repo.data after processing".format(len(repo.data)))
        return repo

In [ ]:
workRepo = load_activity_data_from_files(activity, environment.CONFIG.course)

# This is necessary if using post CAN-44
data = load_activity_data_from_files(activity, environment.CONFIG.course)
workRepo = WorkRepositoryFactory.make( activity, environment.CONFIG.course )
workRepo.data = data
workRepo.set_question_columns( data )
# print( "{} rows loaded to repo.data after processing".format( len( workRepo.data ) ) )

In [ ]:
workRepo.data

In [ ]:
# frames[1]

In [ ]:
EXTRANEOUS_COLUMNS = ['has_seen_results',
       'html_url',  'overdue_and_needs_submission',
       'quiz_points_possible','validation_token',
       'section_id', '_requester', 'attempts_left',
       'end_at', 'end_at_date', 'excused?', 'extra_attempts', 'extra_time',
       'finished_at', 
       'started_at', 'attributes', 'course_id', 
 'manually_unlocked', 'n correct',
       'n incorrect','quiz_version', 'result_url',
       'score_before_regrade', 'section', 'sis_id',
       'started_at_date', 'submitted',  'kept_score'
       'time_spent',  'workflow_state']

In [ ]:
KEEP_COLUMNS = ['attempt', 'course_id', 'finished_at_date',
       'fudge_points', 'id', 'name', 'quiz_id', 'score',
       'section_sis_id', 'student_id', 'submission_id', 'user_id',  'workflow_state']

In [ ]:
[ a for a in f.attempt if a >1]

In [ ]:
f.columns

In [ ]:
j[:3]

In [ ]:
g = f.groupby('student_id')
j = f.set_index(['student_id', 'attempt'])

In [ ]:
r = []
for n, group in g:
#     r.append( j.loc[n, group.attempt.max()])
    r.append((n, group.attempt.max()))
# r = pd.DataFrame(r)

In [ ]:
len(r)

In [ ]:
len(j.loc[r])

In [ ]:
f.loc[r].T

In [ ]:
def s(group):
    latest_attempt = group.attempt.max()
    print(latest_attempt)
#     return 
    
g.filter(lambda x: x.attempt == x.attempt.max())

In [ ]:
f[['student_id', 'user_id', 'attempt']]

In [ ]:
f[:3].T

# Download submission objects

In [ ]:
results = get_all_quiz_submissions(activity.course_id, activity.quiz_id)
results[0]

In [ ]:
# Get all submissions and save them in a dataframe
# We will need the ids when it comes time to upload grades (I think)

submissions = []

for quiz in quizzes:
    results = get_all_quiz_submissions(quiz.course_id, quiz.id)
    for r in results:
        submissions.append({'course_id' : int(quiz.course_id), 
                             'quiz_id' : int(quiz.id),
                             'student_id' : int(r['user_id']), 
                             'submission_id': int(r['id']),
                             'attempt' : int(r['attempt']) 
                            })
submissions = pd.DataFrame(submissions)
len(submissions)

API doesn't allow to download individual student answers. It does allow to download the answers in a csv via the student analysis report.

Log into Canvas, go to the assignment --> Quiz Statistics --> Student analysis

See https://github.com/instructure/canvas-lms/issues/742

# Load student answers from csv files

Before doing this, the Student Analysis reports need to be manually downloaded from canvas and placed in the correct folder!

# Grade based on emptiness

In [ ]:
# This is the current version. Should be able to handle
# first-run-grading or any subsequent run

NO_LATE_PENALTY = True

graded = []
selected_ids = [a[0] for a in environment.CONFIG.assignments]
to_grade = [a for a in environment.CONFIG.unit.components if a.id in selected_ids]

# to_grade = [environment.CONFIG.unit.initial_work]
for activity in to_grade:
    # If don't just want to use lock date for determining 
    # last chance for half credit (e.g., midterm for quarter credit), do this
    # activity.quarter_credit_deadline = QUARTER_CREDIT_DATE
    if NO_LATE_PENALTY:
        activity.due_at = activity.lock_at
    # Get all submission records for the activity
    quiz = environment.CONFIG.course.get_quiz(activity.quiz_id)
    subRepo = QuizSubmissionRepository(quiz)
#     assignment = environment.CONFIG.course.get_assignment(int(activity.id))
    
    # Load all new records since last download
    newframe = load_new(activity)
    len(newframe)
    student_work = process_work(newframe, subRepo.frame)
    remove_non_final_attempts(student_work)
    
    # Load processed work into a quiz repository
    workRepo = QuizRepository(activity, environment.CONFIG.course)
    workRepo.set_question_columns(student_work)
    workRepo.data = student_work
    
    # Let the grader do its job
    grader = QuizGrader(workRepo, subRepo)
    g = grader.grade()
    graded += g
len(graded)

In [ ]:
workRepo.data

In [ ]:
subRepo.frame

# Upload grades

GET /api/v1/courses/:course_id/quizzes/:quiz_id/submissions/:id 
 
PUT /api/v1/courses/:course_id/quizzes/:quiz_id/submissions/:id 
                
{
  "quiz_submissions": [{
    "attempt": 1,
    "fudge_points": -2.4
  }]
}                

In [ ]:
# If this is a quiz type assignment, we need to get the quiz submission object
# not the regular submission object

quiz = environment.CONFIG.course.get_quiz(activity.quiz_id)
qsubs = [s for s in quiz.get_submissions()]

def get_submission_object(student_id, attempt):    
    return [d for d in qsubs if d.user_id == student_id and d.attempt == attempt]

# Upload grades
for g in graded:
    sub = get_submission_object(g['student_id'], g['attempt'])[0]
    sub.update_score_and_comments(quiz_submissions=g['data']['quiz_submissions'])

In [ ]:
def upload_quiz_grades( activity, grade_data ):
    cnt = 0
    for r in grade_data:
        url = make_url( activity.course_id, 'quizzes' )
#         url = make_url( int(r[ 'course_id' ]), 'assignments' )
        url += "/%s/submissions/%s" % (activity.quiz_id, r[ 'submission_id' ])
#         url += "/%s/submissions/%s" % (r[ 'quiz_id' ], r[ 'student_id' ]) #r[ 'submission_id' ])
        print(url)
        response = send_put_request( url, r[ 'data' ] )
        print(response)
        cnt += 1
    print( "{} grades uploaded".format( cnt ) )

# Handle the uploading of grades after a 
# visual sanity check
upload_quiz_grades(activity, graded)
# for g in graded:
#     upload_quiz_grades(g)

# Detect copied answers

# Visualizations

## Combine data from all classes

Each class's data will have different names for the quiz questions. 
Thus before we combine the data frames we need to rename the columns, so it will line up properly

- Todo: Once have a better sense of the variations, rewrite using regular expressions

In [ ]:

def normalize_q_names(txt):
    # remove newlines which make the titles different
    txt = txt.replace('\n', " ")
    # The question title will have the format 000000: text text text text
    # so we keep the main body of the text (we will lose anything after a ':' in the text)
    txt = txt.split(":")[1]
    return txt.strip()
    

for f in frames:
    cols = detect_question_columns(f)
    # Make a dictionary mapping the old names to the new names
    rename_cols = {c: normalize_q_names(c) for c in cols}
    f.rename(rename_cols, axis='columns', inplace=True)


d = pd.concat(frames)

drop_columns_from_frame(d)

d.columns

d.to_excel("%s/oa1.xlsx" %(environment.LOG_FOLDER))


data = {}
for r in journal_folders:
    with open("%s/all-submissions.json" % r, 'r') as f:
        j = json.load(f)
    data[r.split('/')[-1:][0]] = [make_wordbag(str(row['body'])) for row in j]

# data

field_name= "What is an example of persuasive advertising? (Don't use Crisp's examples)"

def make_bag_from_oa_field(frame, field_name):
    bag = []
    bags = [ make_wordbag(str(row)) for i, row in frame[field_name].iteritems()]
    for b in bags:
        bag = bag + b
    return bag



draw_cloud(bag, field_name)

fields = ["What is an example of informational advertising? (Don't use Crisp's examples)",
 'Why does it matter that the desires activated by persuasive advertising are unconscious?',
 'Give an example of informational advertising which Crisp might find objectionable',
 "Suppose that a movie theater projectionist accidentally splices a picture of ice cream into a movie so that it appears for the same amount of time as the alleged subliminal advertising Crisp discusses. If someone in the audience buys ice cream due to this influence, is their choice autonomous on Crisp's account? (Hint'",
 "Write your own reading comprehension question for students in next year's class. Explain the correct answer."]

drop = ['name','student_id', 'sis_id', 'section', 'section_id', 'section_sis_id',
       'submitted', 'attempt','n correct', 'n incorrect', 'score', 'course_id', 'quiz_id',
       'submission_id', 'graded_total', 'level_0', 'index']
# d.reset_index(inplace=True)
c = d.drop(drop, axis=1)

c =c.rename(lambda x : x[:50].strip(), axis=1)
c

bgs = [(f, make_bag_from_oa_field(c, f)) for f in c]
    

for title,bag in bgs:
    draw_cloud(bag, title)

# Attic

These cells are probably ready for deletion

In [ ]:
WAIT =5
NO_LATE_PENALTY =True


# programmatically get data
activity = [c for c in environment.CONFIG.unit.components if c.id == environment.CONFIG.assignments[0][0]][0]
    
if NO_LATE_PENALTY:
    activity.due_at = activity.lock_at

workRepo = WorkRepositoryLoaderFactory.make(course=environment.CONFIG.course, activity=activity, rest_timeout=WAIT)

quiz = environment.CONFIG.course.get_quiz(activity.quiz_id)
subRepo = QuizSubmissionRepository(quiz)

# Filter previously graded
subRepo.data = [s for s in subRepo.data if s.workflow_state != 'complete']
workRepo.data = workRepo.data[workRepo.data.workflow_state != 'complete'].copy(deep=True)

graded = []

workRepo.data.reset_index(inplace=True)
# Let the grader do its job
grader = QuizGrader(workRepo, subRepo)
g = grader.grade(on_empty=0)
graded += g
print("Graded: ", len(graded))

# If this is a quiz type assignment, we need to get the quiz submission object
# not the regular submission object

quiz = environment.CONFIG.course.get_quiz(activity.quiz_id)
qsubs = [s for s in quiz.get_submissions()]

def get_submission_object(student_id, attempt):    
    return [d for d in qsubs if d.user_id == student_id and d.attempt == attempt]

# Upload grades
for g in graded:
    sub = get_submission_object(g['student_id'], g['attempt'])[0]
    sub.update_score_and_comments(quiz_submissions=g['data']['quiz_submissions'])

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!
# Version prior to 2020-02-16
# This handles everything
# Make sure to manually download the reports and
# to download the submission info before running

GRACE_PERIOD = pd.Timedelta('2 days')

graded = []
for quiz in quizzes:
    print("Grading: {}".format(quiz.name))
    # Load all student data for selected assignments from csv files
    fiter = makeDataFileIterator(quiz.folder_path)
    try:
        while True:
            f = next(fiter)
            print("loading: ", f)
            student_work = load_student_work(f, submissions)

            # add submission info and use to remove non-final attempts
            remove_non_final_attempts(student_work)
            # Add some values to the quiz before grading
            quiz.quarter_credit_date = QUARTER_CREDIT_DATE
            quiz.set_question_columns(student_work)
            # Grade it
            g = grade(student_work, quiz, GRACE_PERIOD)
            graded.append(g)
            # Save copy to log folder
            save_json(g, quiz)
    except StopIteration:
        pass

len(graded[0])

# graded

In [ ]:
from CanvasHacks.Repositories.quizzes import process_work, load_student_work, remove_non_final_attempts, make_drop_list, save_json, drop_columns_from_frame

In [ ]:
assignments = []
for quiz in quizzes:
# for folder_path in ASSIGNMENT_FOLDERS:
#     print(quiz.folder_path)
    fiter = makeDataFileIterator(quiz.folder_path)
    try:
        while True:
            f = next(fiter)
            print(f)
            frame = load_student_work(f, submissions)
            assignments.append(frame)

    except:
        pass
len(assignments)

# Load all student data for selected assignments from csv files
assignments = []
for quiz in quizzes:
# for folder_path in ASSIGNMENT_FOLDERS:
    print("Quiz: ", quiz.folder_path)
    fiter = makeDataFileIterator(quiz.folder_path)
#     frame = load_student_work(next(fiter), submissions)
    try:
        while True:
            f = next(fiter)
            print("loading: ", f)
            frame = load_student_work(f, submissions)
            # add submission info and use to remove non-final attempts
            remove_non_final_attempts(frame)
            assignments.append(frame)
    except StopIteration:
        pass

    
#     assignments.append(frame)
len(assignments)

In [ ]:
course_id = sections[0]

quiz_id = environment.CONFIG.assignments[0][0]
results = get_all_quiz_submissions(course_id, quiz_id)
results

In [ ]:

def check_responses(row, question_columns):
    score = 0
    for c in question_columns:
        try:
            if pd.isnull(row[c]): 
                raise Exception
                
            # No credit for 1 word answers
            if len(row[c]) < 2:
                raise Exception
            
            # If we made it past the tests, increment the score
            score += 1
        except Exception:
            pass
                
    return score

def add_graded_total_field(frame, question_columns):
    frame['graded_total'] = frame.apply(lambda r: check_responses(r, question_columns), axis=1)


def save_to_log_folder(frame):
    section = frame.iloc[0]['section']
    qid = frame.iloc[0]['quiz_id']
    fname = "%s/%s-%s-results.xlsx" % (environment.LOG_FOLDER, section, qid)
    print("Saving to ", fname)
    frame.to_excel(fname)

def grade(frame, quiz_data_obj, grace_period=None):
    """
    This handles the actual grading
    
    quiz_data_obj will have the payload format:
        "quiz_submissions": [{
        "attempt": int(attempt),
        "fudge_points": total_score
      },
          "questions": {
      "QUESTION_ID": {
        "score": null, // null for no change, or an unsigned decimal
        "comment": null // null for no change, '' for no comment, or a string
      }
    """
    results = []
#     questions = detect_question_columns(frame.columns)

    for i, row in frame.iterrows():
        fudge_points = 0
        out = {
            'student_id': int(row['student_id']),
            'attempt': int(row['attempt']),
            'submission_id': int(row['submission_id']),
            'course_id': int(row['course_id']),
            'quiz_id': int(row['quiz_id']),
            'data': {}
        }
        # used for computing penalty 
        total_score = 0
        questions = {}

        for qid, column_name in quiz_data_obj.question_columns:
            if pd.isnull(row[column_name]):
                questions[qid] = { 'score': 0}
            else:
                questions[qid] = { 'score': 1.0 }
                total_score += 1
        
        # compute penalty if needed
        penalty = get_penalty(row['submitted'], quiz_data_obj.due_date, quiz_data_obj.quarter_credit_date, grace_period)
        # will be 0 if not docking
        fudge_points = total_score * -penalty
        if penalty >0:
            print('Student #{}: Submitted on {}; was due {}. Penalized {}'.format(row['student_id'], row['submitted'],quiz_data_obj.due_date, penalty))


        out['data']["quiz_submissions"] = [
            { 
                "attempt": int(row['attempt']),
                "fudge_points": fudge_points,
                "questions": questions
            }
        ]

        results.append(out)
    return results


def save_json(grade_data, quiz_data_obj):
    fpath = "%s/%s-%s-all-submissions.json" % (environment.LOG_FOLDER, quiz_data_obj.course_id, quiz_data_obj.id)
    # save submissions
    with open(fpath, 'w') as fpp:
        json.dump(grade_data, fpp)


def upload_quiz_grades(grade_data):
    cnt = 0
    for r in grade_data:
        url = make_url(r['course_id'], 'quizzes')
        url += "/%s/submissions/%s" % (r['quiz_id'], r['submission_id'])
#         print(url)
        response = send_put_request(url, r['data'])
        cnt += 1
    print("{} grades uploaded".format(cnt))

In [ ]:
class QuizGrader:

    def __init__( self, work_repo, submission_repo, grade_func=None):
        """
        :param grade_func: Function or method to use to determine grade
        """
        self.grade_func = grade_func
        self.work_repo = work_repo
        self.submission_repo = submission_repo


    @property
    def activity( self ):
        return self.work_repo.activity

    def grade( self ):
        self.graded = [ ]
        for i, row in self.work_repo.data.iterrows():
            self.graded.append(self._grade_row(row))
        return self.graded

    def _grade_row(self, row):
        fudge_points = 0
        out = {
            'student_id': int( row[ 'student_id' ] ),
            'attempt': int( row[ 'attempt' ] ),
            'submission_id': int( row[ 'submission_id' ] ),
            'course_id': int( row[ 'course_id' ] ),
            'quiz_id': int( row[ 'quiz_id' ] ),
            'data': { }
        }
        # used for computing penalty
        total_score = 0
        questions = { }

        # Grade on emptiness
        # todo This should use credit_no_credit from GradingTools.nonempty
        for qid, column_name in self.work_repo.question_columns:
            if pd.isnull( row[ column_name ] ):
                questions[ qid ] = { 'score': 0 }
            else:
                questions[ qid ] = { 'score': 1.0 }
                total_score += 1

        # compute penalty if needed
        penalty = get_penalty( row[ 'submitted' ], self.activity.due_at, self.activity.lock_at, self.activity.grace_period )
        # will be 0 if not docking
        fudge_points = total_score * -penalty
        if penalty > 0:
            print(self._penalty_message( penalty, row ))

        out[ 'data' ][ "quiz_submissions" ] = [
            {
                "attempt": int( row[ 'attempt' ] ),
                "fudge_points": fudge_points,
                "questions": questions
            }
        ]
        return out

    def _penalty_message( self, penalty, row ):
        stem = 'Student #{}: Submitted on {}; was due {}. Penalized {}'
        return stem.format( row[ 'student_id' ], row[ 'submitted' ], self.activity.due_at, penalty )



In [ ]:
# We'll need the id rather than the assignment_id for quizzes, so 
# we need to replace the assignment ids in the store
quizzes = get_all_course_quizzes(sections[0])

n = []
for a_id, name in environment.CONFIG.assignments:
    for q in quizzes:
        if q['title'] == name:
            n.append((q['id'], name))
#             print(a_id, q['id'], name)

environment.CONFIG.assignments = n


In [ ]:
def get_quiz_questions(course_id, quiz_id):
    """
    GET /api/v1/courses/:course_id/quizzes/:quiz_id/questions 
    """
    url = make_url(course_id, 'quizzes')
    url += "/%s/questions" % quiz_id
    print(url)
    return send_get_request(url)

questions = get_quiz_questions(41179, 100843)

In [ ]:
def get_due_date(quiz_id, quizzes):
    return pd.to_datetime(list(filter(lambda x: x['id'] == quiz_id, quizzes))[0]['due_at'])

def get_lock_date(quiz_id, quizzes):
    return pd.to_datetime(list(filter(lambda x: x['id'] == quiz_id, quizzes))[0]['lock_at'])

In [ ]:
# This stuff was earlier version which didn't work
# The working scripts to grade and upload are below in next section
# Keeping for now, because may be good to use this for backup of data

for frame in frames:
    questions = detect_question_columns(frame.columns)
    add_graded_total_field(frame, questions)
    remove_non_final_attempts(frame)
#     save_to_log_folder(frame)



In [ ]:
def add_penalty(data, max_possible, penalty):
    """data should look like this:
    {'student_id': 1234,
 'attempt': 1,
 'submission_id': 12345,
 'course_id': 12345,
 'quiz_id': 12345,
 'data': {'quiz_submissions': [{'attempt': 1,
    'fudge_points': 0,
    'questions': {'2044306': {'score': 1.0},
     '2282600': {'score': 1.0},
     '2044307': {'score': 1.0},
     '2044308': {'score': 1.0},
     '2044309': {'score': 1.0}}}]}}

"""
    data['data']['quiz_submissions'][0]['fudge_points'] = -max_possible * penalty#['questions']